# SI Regressions
This file loads the BG-level and CT-level datasets and runs the regressions shown in the Supplementary Information.

In [1]:
data_folder = ''
figure_path = ''

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import itertools
from tqdm import tqdm
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'Arial'
from IPython.display import display, Latex, HTML
from stargazer.stargazer import Stargazer, LineLocation

# Read data

In [3]:
# Read BG-level data
df_BG = pd.read_csv(data_folder + '23_level_BG_US_imputedquantiles_20250722.zip', index_col=0)#+ '21c_level_BG_US_compiled_wlabels_250702.zip',index_col=0)
df_BG['STATEFP'] = df_BG['STATEFP'].astype(str).str.zfill(2)  # Ensure STATEFP is two digits
df_BG['COUNTYFP'] = df_BG['COUNTYFP'].astype(str).str.zfill(5)  # Ensure COUNTYFP is three digits
df_BG.index = df_BG.index.astype(str).str.zfill(12)  # Ensure BGFP is six digits

print('Mean size of a BG:', df_BG[df_BG['total_pop_byBG']>0]['total_pop_byBG'].mean())
print('Std:', df_BG[df_BG['total_pop_byBG']>0]['total_pop_byBG'].std())

# Read CT-level data
df_CT = pd.read_csv(data_folder + '23_level_CT_US_imputedquantiles_20250722.zip', index_col='TRACTFP')#+ '22_level_CT_US_compiled_wlabels_250702.zip',index_col='TRACTFP')
df_CT['STATEFP'] = df_CT['STATEFP'].astype(str).str.zfill(2)  # Ensure STATEFP is two digits
df_CT['COUNTYFP'] = df_CT['COUNTYFP'].astype(str).str.zfill(5)  # Ensure COUNTYFP is three digits
df_CT.drop('Unnamed: 0', axis=1, inplace=True)  # Drop unnecessary column
df_CT.index = df_CT.index.astype(str).str.zfill(11)  # Ensure BGFP is six digits

Mean size of a BG: 1374.4094576338239
Std: 705.1132344046625


In [4]:
# Scale columns for regression
income_cols_BG = ['median_household_income_imputed', 'av_income_10km_withoutBG_imputed', 'av_income_50km_withoutBG_imputed', ]
for col in income_cols_BG:
    df_BG[col + '_k'] = df_BG[col] / 1000.0  # Convert income to thousands of USD
    df_BG[col + '_10k'] = df_BG[col] / 10000.0  # Convert income to 10s of thousands of USD
    
income_cols_CT = ['median_household_income_byCT_imputed', 'av_income_10km_withoutCT', 'av_income_50km_withoutCT']
for col in income_cols_CT:
    df_CT[col + '_k'] = df_CT[col] / 1000.0  # Convert income to thousands of USD
    df_CT[col + '_10k'] = df_CT[col] / 10000.0  # Convert income to 10s of thousands of USD
    
df_BG['nearest_highway_km_BG' + '_100km'] = df_BG['nearest_highway_km_BG'] / 100.0 # Convert to hundreds of kms
df_CT['nearest_highway_km_BG' + '_100km'] = df_CT['nearest_highway_km_BG'] / 100.0 # Convert to hundreds of kms

# Functions

In [5]:
# Run a single regression
def run_regression(df_data,dep_var,indep_var,covs=[],state_FE=False,which='OLS'):
    df_data_noNaN = df_data.dropna(subset=[indep_var,dep_var] + covs)
    print(dep_var + ' ~ ' + indep_var + ' + '+ str(covs) + ' + ' + str(state_FE))
    X = df_data_noNaN[[indep_var] + covs]  # Independent variable(s)
    if state_FE:
        assert type(df_data_noNaN['STATEFP'].iloc[0]) == str  # Ensure STATEFP is a string
        dummies = pd.get_dummies(df_data_noNaN['STATEFP'], dtype=int, drop_first=True)
        X = pd.concat([X, dummies], axis=1)
    X = sm.add_constant(X)  # Adds a constant term to the predictor
    Y = df_data_noNaN[dep_var]  # Dependent variable
    if which == 'OLS':
        model = sm.OLS(Y, X).fit()
    elif which == 'Logit':
        model = sm.Logit(Y, X).fit()
    return model

# Table S1: Extended main regression

In [6]:
# Regressions for BG-level data
OLS_corr = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',[],state_FE=None,which='OLS')
OLS_StateFE_imputed = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',[],state_FE='STATEFP',which='OLS')
OLS_BG_10_imputed = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k'],state_FE='STATEFP',which='OLS')
OLS_BG_50_imputed = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',['av_income_50km_withoutBG_imputed_10k'],state_FE='STATEFP',which='OLS')
OLS_BG_hwy = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k','nearest_highway_km_BG_100km'],state_FE='STATEFP',which='OLS')

no_stations ~ median_household_income_imputed_10k + [] + None
no_stations ~ median_household_income_imputed_10k + [] + STATEFP
no_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k'] + STATEFP
no_stations ~ median_household_income_imputed_10k + ['av_income_50km_withoutBG_imputed_10k'] + STATEFP
no_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k', 'nearest_highway_km_BG_100km'] + STATEFP


In [7]:
list_covs_cleaned = ['median_household_income_imputed_10k', 
                     'av_income_10km_withoutBG_imputed_10k', 
                     'av_income_50km_withoutBG_imputed_10k',
                     'nearest_highway_km_BG_100km'] # Hide state fixed effects (if implemented as fixed effects labels)

dict_rename_covs = {'median_household_income_imputed_10k': 'BG median household income [10kUSD]',
                    'av_income_10km_withoutBG_imputed_10k':'Average neighbouring income [10kUSD], BG/10km',
                    'av_income_50km_withoutBG_imputed_10k':'Average neighbouring income [10kUSD], BG/50km',
                    'nearest_highway_km_BG_100km':'Distance to nearest highway [100km]'}

In [8]:
stargazer = Stargazer([OLS_corr, OLS_StateFE_imputed, OLS_BG_10_imputed, OLS_BG_50_imputed, OLS_BG_hwy])
stargazer.significance_levels([0.05, 0.01, 0.001])
stargazer.dependent_variable_name('Number of stations')
stargazer.covariate_order(list_covs_cleaned)
stargazer.rename_covariates(dict_rename_covs)

stargazer.significant_digits(3)
stargazer.show_degrees_of_freedom(False)

stargazer.add_line('Unit of observation', ['BG', 'BG', 'BG', 'BG','BG'], LineLocation.FOOTER_TOP)
stargazer.add_line('State FE', ['No', 'Yes', 'Yes', 'Yes','Yes'], LineLocation.FOOTER_TOP)

stargazer

In [9]:
htmlstring = stargazer.render_html()
with open(figure_path + "Figures/Table_S1.html", "w") as file:
    file.write(htmlstring)
    
# Generate LaTeX output
print(stargazer.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{\textit{Dependent variable: Number of stations}} \
\cr \cline{2-6}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 BG median household income [10kUSD] & 0.012$^{***}$ & 0.003$^{***}$ & -0.012$^{***}$ & -0.003$^{***}$ & -0.012$^{***}$ \\
& (0.001) & (0.001) & (0.001) & (0.001) & (0.001) \\
 Average neighbouring income [10kUSD], BG/10km & & & 0.056$^{***}$ & & 0.056$^{***}$ \\
& & & (0.002) & & (0.002) \\
 Average neighbouring income [10kUSD], BG/50km & & & & 0.054$^{***}$ & \\
& & & & (0.003) & \\
 Distance to nearest highway [100km] & & & & & -0.017$^{}$ \\
& & & & & (0.013) \\
\hline \\[-1.8ex]
 Unit of observation & BG & BG & BG & BG & BG \\
 State FE & No & Yes & Yes & Yes & Yes \\
 Observations & 239780 & 239780 & 239766 & 239776 & 239766 \\
 $R^2$ & 0.001 & 0.010 & 0.013 & 0.012 & 0.013 \\
 Adjusted $R^2$ & 0.001 & 0.010 & 0.013 & 0.012

# Table S2

In [10]:
OLS_corr = run_regression(df_CT,'no_stations','median_household_income_byCT_imputed_10k',[],state_FE=None,which='OLS')
OLS_StateFE = run_regression(df_CT,'no_stations','median_household_income_byCT_imputed_10k',[],state_FE='STATEFP',which='OLS')
OLS_CT_10 = run_regression(df_CT,'no_stations','median_household_income_byCT_imputed_10k',['av_income_10km_withoutCT_10k'],state_FE='STATEFP',which='OLS')
OLS_CT_50 = run_regression(df_CT,'no_stations','median_household_income_byCT_imputed_10k',['av_income_50km_withoutCT_10k'],state_FE='STATEFP',which='OLS')
OLS_CT_hwy = run_regression(df_CT,'no_stations','median_household_income_byCT_imputed_10k',['av_income_10km_withoutCT_10k','nearest_highway_km_BG_100km'],state_FE='STATEFP',which='OLS')

no_stations ~ median_household_income_byCT_imputed_10k + [] + None
no_stations ~ median_household_income_byCT_imputed_10k + [] + STATEFP
no_stations ~ median_household_income_byCT_imputed_10k + ['av_income_10km_withoutCT_10k'] + STATEFP
no_stations ~ median_household_income_byCT_imputed_10k + ['av_income_50km_withoutCT_10k'] + STATEFP
no_stations ~ median_household_income_byCT_imputed_10k + ['av_income_10km_withoutCT_10k', 'nearest_highway_km_BG_100km'] + STATEFP


In [11]:
list_covs_cleaned = ['median_household_income_byCT_imputed_10k', 
                     'av_income_10km_withoutCT_10k', 
                     'av_income_50km_withoutCT_10k',
                     'nearest_highway_km_BG_100km'] # Hide state fixed effects (if implemented as fixed effects labels)

dict_rename_covs = {'median_household_income_byCT_imputed_10k': 'CT median household income [10kUSD]',
                    'av_income_10km_withoutCT_10k':'Average neighbouring income [10kUSD], BG/10km',
                    'av_income_50km_withoutCT_10k':'Average neighbouring income [10kUSD], BG/50km',
                    'nearest_highway_km_BG_100km':'Distance to nearest highway [100km]'}

In [12]:
stargazer = Stargazer([OLS_corr,OLS_StateFE,OLS_CT_10,OLS_CT_50,OLS_CT_hwy])
stargazer.significance_levels([0.05, 0.01, 0.001])
stargazer.dependent_variable_name('Number of stations')
stargazer.covariate_order(list_covs_cleaned)
stargazer.rename_covariates(dict_rename_covs)

stargazer.significant_digits(3)
stargazer.show_degrees_of_freedom(False)

stargazer.add_line('Unit of observation', ['CT', 'CT', 'CT', 'CT','CT'], LineLocation.FOOTER_TOP)
stargazer.add_line('State FE', ['No', 'Yes', 'Yes', 'Yes','Yes'], LineLocation.FOOTER_TOP)

stargazer

In [13]:
htmlstring = stargazer.render_html()
with open(figure_path + "Figures/Table_S2.html", "w") as file:
    file.write(htmlstring)
    
print(stargazer.render_latex())  # For LaTeX

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{\textit{Dependent variable: Number of stations}} \
\cr \cline{2-6}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 CT median household income [10kUSD] & 0.055$^{***}$ & 0.026$^{***}$ & -0.025$^{***}$ & 0.004$^{}$ & -0.025$^{***}$ \\
& (0.003) & (0.003) & (0.004) & (0.003) & (0.004) \\
 Average neighbouring income [10kUSD], BG/10km & & & 0.157$^{***}$ & & 0.157$^{***}$ \\
& & & (0.007) & & (0.007) \\
 Average neighbouring income [10kUSD], BG/50km & & & & 0.147$^{***}$ & \\
& & & & (0.009) & \\
 Distance to nearest highway [100km] & & & & & -0.025$^{}$ \\
& & & & & (0.041) \\
\hline \\[-1.8ex]
 Unit of observation & CT & CT & CT & CT & CT \\
 State FE & No & Yes & Yes & Yes & Yes \\
 Observations & 84414 & 84414 & 84404 & 84411 & 84404 \\
 $R^2$ & 0.004 & 0.025 & 0.031 & 0.028 & 0.031 \\
 Adjusted $R^2$ & 0.004 & 0.024 & 0.030 & 0.027 & 0.030 

# Table S3

In [14]:
# Include has_stations
df_BG['has_stations'] = 0
df_BG.loc[df_BG['no_stations'] > 0,'has_stations'] = 1

In [15]:
# BGs with stations
df_BG_hasstations = df_BG.loc[df_BG['has_stations'] == 1]

In [16]:
# Regressions for BG-level data
OLS_BG_10 = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k'],state_FE='STATEFP',which='OLS')
OLS_BG_atleastone = run_regression(df_BG_hasstations,'no_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k'],state_FE='STATEFP',which='OLS')
OLS_BG_L2 = run_regression(df_BG,'no_L2_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k'],state_FE='STATEFP',which='OLS')
OLS_BG_DC = run_regression(df_BG,'no_DC_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k'],state_FE='STATEFP',which='OLS')
OLS_BG_hasstations = run_regression(df_BG,'has_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k'],state_FE='STATEFP',which='Logit')

no_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k'] + STATEFP
no_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k'] + STATEFP
no_L2_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k'] + STATEFP
no_DC_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k'] + STATEFP
has_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k'] + STATEFP
Optimization terminated successfully.
         Current function value: 0.320583
         Iterations 7


In [17]:
list_covs_cleaned = ['median_household_income_imputed_10k', 'av_income_10km_withoutBG_imputed_10k']
dict_rename_covs = {'median_household_income_imputed_10k': 'BG median household income [10kUSD]',
                    'av_income_10km_withoutBG_imputed_10k':'Average neighbouring income [10kUSD], BG/10km'}

In [18]:
stargazer = Stargazer([OLS_BG_10,OLS_BG_atleastone,OLS_BG_L2,OLS_BG_DC,OLS_BG_hasstations])
stargazer.significance_levels([0.05, 0.01, 0.001])
stargazer.dependent_variable_name('Number of stations')
stargazer.covariate_order(list_covs_cleaned)
stargazer.rename_covariates(dict_rename_covs)

stargazer.significant_digits(3)
stargazer.show_degrees_of_freedom(False)

stargazer.add_line('Unit of observation', ['BG', 'BG', 'BG', 'BG','BG'], LineLocation.FOOTER_TOP)
stargazer.add_line('State FE', ['Yes', 'Yes', 'Yes', 'Yes','Yes'], LineLocation.FOOTER_TOP)

stargazer

In [19]:
htmlstring = stargazer.render_html()

with open(figure_path + "Figures/Table_S3.html", "w") as file:
    file.write(htmlstring)
    
print(stargazer.render_latex())  # For LaTeX

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 BG median household income [10kUSD] & -0.012$^{***}$ & -0.016$^{}$ & -0.012$^{***}$ & -0.012$^{***}$ & -0.034$^{***}$ \\
& (0.001) & (0.009) & (0.001) & (0.001) & (0.002) \\
 Average neighbouring income [10kUSD], BG/10km & 0.056$^{***}$ & 0.177$^{***}$ & 0.056$^{***}$ & 0.056$^{***}$ & 0.123$^{***}$ \\
& (0.002) & (0.016) & (0.002) & (0.002) & (0.004) \\
\hline \\[-1.8ex]
 Unit of observation & BG & BG & BG & BG & BG \\
 State FE & Yes & Yes & Yes & Yes & Yes \\
 Observations & 239766 & 24660 & 239766 & 239766 & 239766 \\
 $R^2$ & 0.013 & 0.031 & 0.013 & 0.013 &  \\
 Adjusted $R^2$ & 0.013 & 0.029 & 0.013 & 0.013 &  \\
 Pseudo $R^2$ &  &  &  &  & 0.032 \\
 Residual Std. Error & 1.640 & 4.541 & 1.640 & 1.640 &  \\
 F Statistic & 62.087$^{***}$ & 14.939$^{***}$ & 62.087$^{***}$ & 62.087$^{***}$ &  \\
\hline
\hline \\

# Table S4

In [20]:
df_BG['pop_density_byBG'] = df_BG['total_pop_byBG'] / df_BG['BGALAND']

In [21]:
# Include has_hwy_passing
df_BG['has_highway_passing'] = 0
df_BG.loc[df_BG['nearest_highway_km_BG'] == 0,'has_highway_passing'] = 1

In [22]:
median_density = df_BG['pop_density_byBG'].median()
df_BG_urban = df_BG.loc[df_BG['pop_density_byBG'] > median_density]
df_BG_rural = df_BG.loc[df_BG['pop_density_byBG'] <= median_density]

In [23]:
OLS_BG_10 = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k'],state_FE='STATEFP',which='OLS')
OLS_BG_10_hwy = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k','has_highway_passing'],state_FE='STATEFP',which='OLS')
OLS_BG_10_dens = run_regression(df_BG,'no_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k','pop_density_byBG'],state_FE='STATEFP',which='OLS')
OLS_BG_urban = run_regression(df_BG_urban,'no_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k'],state_FE='STATEFP',which='OLS')
OLS_BG_rural = run_regression(df_BG_rural,'no_stations','median_household_income_imputed_10k',['av_income_10km_withoutBG_imputed_10k'],state_FE='STATEFP',which='OLS')

no_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k'] + STATEFP
no_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k', 'has_highway_passing'] + STATEFP
no_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k', 'pop_density_byBG'] + STATEFP
no_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k'] + STATEFP
no_stations ~ median_household_income_imputed_10k + ['av_income_10km_withoutBG_imputed_10k'] + STATEFP


In [24]:
list_covs_cleaned = ['median_household_income_imputed_10k', 'av_income_10km_withoutBG_imputed_10k', 'has_highway_passing', 'pop_density_byBG']

dict_rename_covs = {'median_household_income_imputed_10k': 'BG median household income [10kUSD]',
                    'av_income_10km_withoutBG_imputed_10k':'Average neighbouring income [10kUSD], BG/10km',
                    'has_highway_passing': 'Has highway passing through',
                    'pop_density_byBG':'Population density'}

In [25]:
stargazer = Stargazer([OLS_BG_10,OLS_BG_10_hwy,OLS_BG_10_dens,OLS_BG_urban,OLS_BG_rural])
stargazer.significance_levels([0.05, 0.01, 0.001])
stargazer.dependent_variable_name('Number of stations')
stargazer.covariate_order(list_covs_cleaned)
stargazer.rename_covariates(dict_rename_covs)
stargazer.significant_digits(3)
stargazer.show_degrees_of_freedom(False)

stargazer.add_line('Unit of observation', ['BG', 'BG', 'BG', 'BG', 'BG'], LineLocation.FOOTER_TOP)
stargazer.add_line('State FE', ['Yes', 'Yes', 'Yes', 'Yes', 'Yes'], LineLocation.FOOTER_TOP)

stargazer

In [26]:
htmlstring = stargazer.render_html()
with open(figure_path + "Figures/Table_S4.html", "w") as file:
    file.write(htmlstring)
    
print(stargazer.render_latex())  # For LaTeX

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{\textit{Dependent variable: Number of stations}} \
\cr \cline{2-6}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 BG median household income [10kUSD] & -0.012$^{***}$ & -0.011$^{***}$ & -0.013$^{***}$ & -0.005$^{***}$ & -0.034$^{***}$ \\
& (0.001) & (0.001) & (0.001) & (0.001) & (0.002) \\
 Average neighbouring income [10kUSD], BG/10km & 0.056$^{***}$ & 0.054$^{***}$ & 0.058$^{***}$ & 0.037$^{***}$ & 0.108$^{***}$ \\
& (0.002) & (0.002) & (0.002) & (0.002) & (0.004) \\
 Has highway passing through & & 0.406$^{***}$ & & & \\
& & (0.008) & & & \\
 Population density & & & -8.352$^{***}$ & & \\
& & & (0.617) & & \\
\hline \\[-1.8ex]
 Unit of observation & BG & BG & BG & BG & BG \\
 State FE & Yes & Yes & Yes & Yes & Yes \\
 Observations & 239766 & 239766 & 239167 & 119589 & 119578 \\
 $R^2$ & 0.013 & 0.023 & 0.014 & 0.012 & 0.027 \\
 Adjusted 